In [ ]:
# test the column transformer
# num cols have mean=0 and std=1
# x = preprocessor.fit_transform(df)
# df_trans = pd.DataFrame(x, columns=preprocessor.get_feature_names_out())
# display(df_trans.describe())
# print(df_trans.shape)

steps = [("preprocessor", preprocessor), ("clf", RandomForestRegressor(max_depth=5))]

rf_pipeline = Pipeline(steps)

# Cross-validate the model
cross_val_scores = cross_val_score(rf_pipeline, X, y, cv=10, scoring="neg_mean_squared_error")

# Print the 10-fold RMSE
print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

rf_pipeline.fit(X, y)
# Get feature names after preprocessing
feature_names = rf_pipeline.named_steps["preprocessor"].get_feature_names_out()
# print(feature_names)
feature_importance = rf_pipeline.named_steps["clf"].feature_importances_


df = pd.DataFrame({"feature": feature_names, "importance": feature_importance}).sort_values(
    by="importance", ascending=False
)

# Plot
plt.figure(figsize=(10, 6))
plt.barh(df["feature"], df["importance"], color="skyblue")
plt.xlabel("Mean Feature Importance")
plt.title("Random Forest Regressor Feature Importances (Cross-Validated)")
plt.gca().invert_yaxis()  # Highest at top
plt.tight_layout()
# plt.show()
plt.savefig("feature_importances.png")  # Save the figure as a PNG image

# Save the fitted pipeline
joblib.dump(rf_pipeline, "./saved_models/model_pipeline.pkl")

# Later: Load it back
loaded_pipeline = joblib.load("./saved_models/model_pipeline.pkl")

# Use it for prediction
# y_pred = loaded_pipeline.predict(X_new)

In [49]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import mean_squared_error

TARGET_COL = "SalePrice"
TRAIN_PATH = "../processed_data/train_processed.csv"
TEST_PATH = "../processed_data/test_processed.csv"
MAX_DEPTH = 6

def load_dataset(filepath:str, target_col: str):
    df = pd.read_csv(filepath)
    X = df.drop(columns=[target_col])
    y = df[target_col]

    return X, y

if __name__ == "__main__":
    # load the training data
    X_train, y_train = load_dataset(TRAIN_PATH, TARGET_COL)
    print(df_train.shape)

    # Load the test data
    X_test, y_test = load_dataset(TEST_PATH, TARGET_COL)
    df_test = pd.read_csv("../processed_data/test_processed.csv")
    
    rf = RandomForestRegressor(max_depth=MAX_DEPTH)

    # Get the accuracy on the training data using cross validation
    cross_val_scores = cross_val_score(rf,X_train,y_train,cv=5,scoring="neg_mean_squared_error")
    # Print the 10-fold RMSE
    print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

    # Fit the model on the toal training data
    rf.fit(X_train, y_train)
    
    # Determine the accuracy on the test dataset
    y_pred = rf.predict(X_test)
    neg_mse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Negative MSE:", neg_mse)

    # TODO save the model

(1168, 62)
10-fold RMSE:  33911.20728674929
Negative MSE: 35901.660798385434
